In [42]:
import sys
sys.path.append("../../src")

import numpy as np
import pandas as pd

import lightgbm

from utils import ndcg_calculator
from LGBM_Rank import LGBMRank
from dataload import dataload, day_week_feature_engineering, train_label_split, feature_dataload
from feature import *
from tqdm import tqdm

In [43]:
import os
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
    # When Running on the CuDNN backend, two further options muset be set
#     torch.backends.cudnn.deteministic =True
#     torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed()

# dataload
- 편의상 df_train_week 변수 통일하여 사용함

In [44]:
path='../../data/'

test_answer_week, test_answer_month, \
df_train_week, df_train_month, \
sample_sumbission_week, sample_sumbission_month = dataload(path)

In [45]:
# # # 전체 데이터 이용시
# df_history = pd.read_csv(path+'history_data.csv')

# # ## 날짜 전처리
# df_history = df_history.assign(log_dt = pd.to_datetime(df_history.log_time//100, format="%Y%m%d%H%M"))
# df_history = df_history.assign(log_date = df_history.log_dt.dt.floor("D"))
# df_history = df_history.drop("log_time", axis=1)

# df_train_week = df_history.copy()

# # # month 이용
# # # df_train_week = df_train_month.copy()

# feature dataload

In [46]:
# data load
watch_df, buy_df, search_df, meta_df, profile_df = feature_dataload(path)

../../src/dataload.py:87: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  meta_df = pd.read_csv(path + "meta_data.csv")


In [47]:
# feature engineering
df_train_week = history_feature_engineering(df_train_week)

short_con_freq_feature = df_train_week[["album_id","short_trailer","continuous_play","album_viewcount_freq"]]

watch_df_feature = watch_feature_engineering(watch_df)
watch_df_feature = watch_df_feature.drop_duplicates(subset=["profile_id","album_id"])

paid_df_feature = paid_feature_engineering(df_train_week, buy_df)

searched_df_feature = searched_feature_engineering(df_train_week, search_df)

meta_df_feature = meta_feature_engineering(meta_df)
meta_df_feature = meta_df_feature.drop(columns="sub_title").drop_duplicates()

profile_df_feature = profile_feature_engineering(profile_df)
profile_df_feature = profile_df_feature[["profile_id","sex","age","age_bin"]]

# day_week feature engineering
- to split label, train data

In [48]:
df_train_week = day_week_feature_engineering(df_train_week)

log date min: 2022-03-01 00:00:00 log date max: 2022-04-23 00:00:00
min day: 0 max day: 53
min week: 0 max week: 7


# label & train split

In [49]:
# Label, train data split
df_train, label_df = train_label_split(df_train_week)

split last week: 7


In [50]:
n = 25

# prerpocess

In [51]:
# 총 3개 파일 사용

# log dt 기준 duplicates
df_train_week = df_train_week.drop_duplicates(subset=["profile_id","album_id","log_dt"])
# ss_id 기준 duplicates
personal_train = df_train.drop_duplicates(subset=['profile_id','album_id','ss_id'])
# album_id, profile_id 기준으로 duplicates
df_train = df_train.drop_duplicates(subset=['profile_id','album_id'])

# 라벨 유저 한정1
# customers = label_df.profileQ_id.unique()
# 전체 유저 한정
customers = df_train.profile_id.unique()

# day week features

In [52]:
# day week feature
interaction_day_week_first = df_train_week[["profile_id","album_id","week","day"]].drop_duplicates(subset=["profile_id","album_id"],keep="first")
interaction_day_week_last = df_train_week[["profile_id","album_id","week","day"]].drop_duplicates(subset=["profile_id","album_id"],keep="last")

In [53]:
# hour_feature

hour_feature = df_train_week[["profile_id","album_id","log_dt"]]
hour_feature["hour"] = hour_feature["log_dt"].apply(lambda x: x.hour)
hour_dummies = pd.get_dummies(hour_feature["hour"])
hour_dummies.columns = ["hour_"+str(i) for i in hour_dummies.columns]
hour_feature = pd.concat([hour_feature, hour_dummies], axis=1)
hour_feature["hour"] = hour_feature["hour"].apply(lambda x: x/x if x !=0 else 0)
hour_feature = hour_feature.groupby(["profile_id","album_id"]).sum().reset_index()

# ratio_hour_feature

ratio_hour_feature = hour_feature.copy()
for col in ratio_hour_feature.columns[3:]:
    ratio_hour_feature[col] = ratio_hour_feature[col]/ratio_hour_feature["hour"]
    
for col in ratio_hour_feature.columns[3:]:
    ratio_hour_feature = ratio_hour_feature.rename(columns={col:"ratio_"+col})
del ratio_hour_feature["hour"]
del hour_feature["hour"]

# hour feature & ratio_hour_feature merge
hour_feature = pd.merge(hour_feature,ratio_hour_feature, how="left", on=["profile_id","album_id"])

/var/folders/tb/j9qmrzv54k73r5l100jxqll80000gn/T/ipykernel_1372/2295550313.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hour_feature["hour"] = hour_feature["log_dt"].apply(lambda x: x.hour)


# MP

In [54]:
MP_df = df_train.album_id.value_counts().head(50).reset_index()
MP_df.columns = ['album_id','total_counts']
MP_df['join_col'] = 1

customer_df = df_train_week[['profile_id']].drop_duplicates()
customer_df['join_col'] = 1

MP_cand = customer_df.merge(MP_df, on='join_col').drop_duplicates(subset=['profile_id','album_id'])[['profile_id','album_id','total_counts']]

# general MP
- 마지막 1주, 2주의 MP를 각 유저마다 넣는다.

In [55]:
last_week_list = np.sort(df_train.week.unique())
print('중복제거 후 데이터 수:', len(df_train_week))

# 마지막 6,5주 각각 MP를 10개 뽑음
last_week_ver1 = last_week_list[-1]
last_week_ver2 = last_week_list[-2]

중복제거 후 데이터 수: 810411


In [56]:
# 마지막 6주
MP_latest_ver1_df = df_train.query(f"week == {last_week_ver1}")

MP_df = MP_latest_ver1_df.groupby('album_id')['profile_id'].count().sort_values(ascending=False)
MP_df = MP_df.reset_index()
MP_df.columns = ['album_id','counts']
MP_candidate_df = MP_df[:10].copy()
MP_candidate_df['join_col'] = 1

# df_train_week 전체 유저 대상으로 후보군을 뽑을 것임
customer_df = df_train_week[df_train_week['profile_id'].isin(customers)][['profile_id']]
customer_df['join_col'] = 1
popular_articles_cand_ver1 = customer_df.copy()
popular_articles_cand_ver1 = popular_articles_cand_ver1.merge(MP_candidate_df, on="join_col")

popular_articles_cand_ver1.drop_duplicates(subset=['profile_id','album_id'],inplace=True)

In [57]:
# 마지막 5주

MP_latest_ver2_df = df_train.query(f"week == {last_week_ver2}")

MP_df = MP_latest_ver2_df.groupby('album_id')['profile_id'].count().sort_values(ascending=False)
MP_df = MP_df.reset_index()
MP_df.columns = ['album_id','general_counts']
MP_candidate_df = MP_df[:10].copy()
MP_candidate_df['join_col'] = 1

customer_df = df_train_week[df_train_week['profile_id'].isin(customers)][['profile_id']]
customer_df['join_col'] = 1
popular_articles_cand_ver2 = customer_df.copy()
popular_articles_cand_ver2 = popular_articles_cand_ver2.merge(MP_candidate_df, on="join_col")

popular_articles_cand_ver2.drop_duplicates(subset=['profile_id','album_id'],inplace=True)

In [58]:
# 마지막 6,5주 concat
popular_articles_cand = pd.concat([popular_articles_cand_ver1, popular_articles_cand_ver2])
popular_articles_cand = popular_articles_cand.groupby(['profile_id','album_id'])['general_counts'].sum().reset_index()

# personal_MP

In [59]:
personal_MP_df = personal_train.groupby(['profile_id','album_id'])[['ss_id']].count().reset_index()
personal_MP_df.columns = ['profile_id','album_id','personal_counts']

In [60]:
# 서로 다른날 5회 이상 시청한 앨범만
personal_MP = personal_MP_df[personal_MP_df['personal_counts'] >= 5]
personal_MP = personal_MP.sort_values(by=['profile_id','personal_counts'],ascending=False)

In [61]:
# 상위 5개만 pick
head_df_list = []
# 전체 유저 대상으로 뽑기
for user_id in tqdm(customer_df.profile_id.unique()):
    personal_MP_user_len = len(personal_MP[personal_MP['profile_id']==user_id].head())
    random_choice_list = personal_MP.album_id.unique()
    if personal_MP_user_len <5:
        # 5개 아이템이 없는 경우 랜덤으로 없는 개수만 만큼 choice
        user_df = personal_MP[personal_MP['profile_id']==user_id]
        df = pd.DataFrame()
        random_choices = np.random.choice(random_choice_list, size=(5-personal_MP_user_len))
        df['profile_id'] = [user_id for _ in range(5-personal_MP_user_len)]
        df['album_id'] = random_choices
        df = pd.concat([user_df, df])
        head_df_list.append(df)
    else:
        head_df_list.append(personal_MP[personal_MP['profile_id']==user_id].head())
        
personal_MP_candidate = pd.concat(head_df_list)

100%|█████████████████████████████████████| 8057/8057 [00:06<00:00, 1256.08it/s]


# MP_user_genre

In [62]:
# meta data load
meta_df = pd.read_csv(path+'meta_data.csv')
meta_df = meta_df[['album_id','genre_mid','run_time','cast_1','cast_2','cast_3']]

df_train_meta = df_train.merge(meta_df, on='album_id')

# 유저 선호 장르별 MP
user_genre_df = df_train_meta.groupby(['profile_id','genre_mid']).count()['ss_id'].reset_index()
user_genre_df.columns = ['profile_id','genre_mid','genre_cnt']
user_genre_df = user_genre_df.groupby(['profile_id','genre_mid']).sum().reset_index().sort_values(by=['profile_id','genre_cnt'],ascending=False)

# 장르 선호도 피처 만들기
## 100이상 시청한 사람들만 percent
user_total_watch_dict = user_genre_df.groupby('profile_id')['genre_cnt'].sum()\
                        [user_genre_df.groupby('profile_id')['genre_cnt'].sum()>=100].to_dict()
# 전체 시청 피처 만들기
user_genre_df['user_genre_cnt'] = user_genre_df['profile_id'].apply(lambda x: user_total_watch_dict.get(x, None))
user_genre_df['user_genre_percent'] = user_genre_df['genre_cnt']/user_genre_df['user_genre_cnt']
user_genre_df.drop(columns=['user_genre_cnt'],inplace=True)
user_genre_df.dropna(subset=['user_genre_percent'],axis=0,inplace=True)

# 장르별 탑 아이템 dict 담기
genre_top_items = {}
genre_count = df_train_meta['genre_mid'].value_counts()
for genre in genre_count.index:
    genre_top_items[genre] = list(df_train_meta[df_train_meta['genre_mid']==genre]['album_id'].value_counts().head(10).index)

/var/folders/tb/j9qmrzv54k73r5l100jxqll80000gn/T/ipykernel_1372/585885348.py:2: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  meta_df = pd.read_csv(path+'meta_data.csv')


In [63]:
# 모든 유저 선호 장르 MP candidate
# 선호장르가 없을 경우 Top 장르인 "노래율동", "TV만화" 장르 MP candidate
df_list = []
for user_id in customer_df.profile_id.unique():
    user_genres = user_genre_df[user_genre_df['profile_id']== user_id].head(2)['genre_mid']
    
    df = pd.DataFrame()
    if len(user_genres) == 0:
        
        df['album_id'] = genre_top_items['노래율동']
        df['album_id'] = genre_top_items['TV만화']
        
    elif len(user_genres) == 1:
        genre_list_1 = genre_top_items[user_genres.values[0]]
        genre_list_2 = genre_top_items['노래율동']
        df['album_id'] = list(dict.fromkeys(np.append(genre_list_1,genre_list_2)))
        
    elif len(user_genres) == 2:
        genre_list_1 = genre_top_items.get(user_genres.values[0],[])
        genre_list_2 = genre_top_items.get(user_genres.values[1],[])
        df['album_id'] = list(dict.fromkeys(np.append(genre_list_1,genre_list_2)))

    df['profile_id'] = user_id
    df_list.append(df)
    
genre_candidate = pd.concat(df_list, ignore_index=True)
genre_candidate = genre_candidate[['profile_id','album_id']]

# ALS MF candidate

In [64]:
from ALS_MF import MF
model = MF(df_train_week)
als_candidate, item_factors_feature, user_factors_feature = model.mf_train()

  0%|          | 0/3 [00:00<?, ?it/s]

item vector shape: (20383, 200) user vector shape (8057, 200)


In [65]:
# def user_item_maps(df):
#     ALL_USERS = df['profile_id'].unique().tolist()
#     ALL_ITEMS = df['album_id'].unique().tolist()

#     idx2user = dict(list(enumerate(ALL_USERS)))
#     idx2item = dict(list(enumerate(ALL_ITEMS)))

#     user2idx = {u: uidx for uidx, u in idx2user.items()}
#     item2idx = {i: iidx for iidx, i in idx2item.items()}

#     df['profile_id'] = df['profile_id'].map(user2idx)
#     df['album_id'] = df['album_id'].map(item2idx)
#     return ALL_USERS, ALL_ITEMS, idx2user, idx2item, user2idx, item2idx, df

# def make_csr_matrix(df):
    
#     ALL_USERS, ALL_ITEMS, idx2user, idx2item, user2idx, item2idx, df = user_item_maps(df)
#     row = df['profile_id'].values
#     col = df['album_id'].values
#     data = np.ones(df.shape[0])
#     csr_train = csr_matrix((data, (row, col)), shape=(len(ALL_USERS), len(ALL_ITEMS)))
#     return csr_train, df

In [66]:
# ALL_USERS, ALL_ITEMS, idx2user, idx2item, user2idx, item2idx, df= user_item_maps(df_train_week[["profile_id","album_id"]].drop_duplicates())

In [67]:
# from scipy.sparse import csr_matrix
# csr_train,df = make_csr_matrix(df_train_week[["profile_id","album_id"]].drop_duplicates())
# matrix = csr_train.todense()

In [68]:
# from sklearn.metrics.pairwise import cosine_similarity
# user_cos_feature = pd.DataFrame(cosine_similarity(matrix, matrix))
# user_cos_feature.columns = ["user_idx_" + str(num) for num in user_cos_feature.columns]
# user_cos_feature = user_cos_feature.reset_index()
# user_cos_feature["index"] = user_cos_feature["index"].apply(lambda x: idx2user[x])
# user_cos_feature = user_cos_feature.rename(columns={"index":"profile_id"})
# del matrix, csr_train, df

# apriori candidaate

In [69]:
from apriori import apriori_train, apriori_candidate
rules_confidence_item_week = apriori_train(df_train_week, 0.1, 0.8)
apriori_candidate, apriori_feature = apriori_candidate(df_train_week, rules_confidence_item_week)

# candidate merge

In [70]:
popular_articles_cand.album_id.nunique(), personal_MP_candidate.album_id.nunique(), genre_candidate.album_id.nunique()

(17, 4715, 167)

In [71]:
candidate_0 = MP_cand
candidate_1 = popular_articles_cand[['profile_id','album_id']]
candidate_2 = personal_MP_candidate[['profile_id','album_id']]
candidate_3 = genre_candidate[['profile_id','album_id']]
candidate_4 = als_candidate.copy()
candidate_5 = apriori_candidate.copy()

# cand = pd.concat([candidate_0, candidate_1, candidate_2, candidate_3, candidate_4, candidate_5])
cand = pd.concat([candidate_0,candidate_1, candidate_2, candidate_4, candidate_5])
# cand = candidate_0.copy()
cand.drop_duplicates(subset=['profile_id','album_id'],inplace=True)

print('cand 데이터 수:',len(cand))
print('cand nunique: ', cand.album_id.nunique())

cand 데이터 수: 1164550
cand nunique:  8687


# features merge

In [72]:
# general counts feature merge
candidate = pd.merge(cand, popular_articles_cand[['album_id','general_counts']].drop_duplicates(), how='left', on='album_id')
# personal_count feature merge
candidate = pd.merge(candidate,personal_MP_df, how='left', on=['profile_id','album_id'])
# apriori feature merge
candidate = pd.merge(candidate, apriori_feature, how="left", on=["profile_id","album_id"])

In [73]:
print('cand 데이터 수:',len(candidate))

cand 데이터 수: 1164550


In [74]:
# meta feature merge
# profile_df = pd.read_csv(path+'profile_data.csv')

'''
아직 사용 안하고 있음
# profile feature add
candidate_add_features = pd.merge(candidate,profile_df, how='left', on='profile_id')
# meta feature add
candidate_add_features = pd.merge(candidate_add_features, meta_df.drop_duplicates('album_id'), how='left', on='album_id')

# catetegory feature label encoding

columns = ['sex','pr_interest_keyword_cd_1','pr_interest_keyword_cd_2','pr_interest_keyword_cd_3','ch_interest_keyword_cd_1','ch_interest_keyword_cd_2','ch_interest_keyword_cd_3',\
'genre_mid','cast_1','cast_2','cast_3']
from sklearn.preprocessing import LabelEncoder
for col in columns:
    LE = LabelEncoder()
    candidate_add_features[col] = LE.fit_transform(candidate_add_features[col])

# drop columns

candidate_add_features = candidate_add_features.drop(columns=['personal_counts','pr_interest_keyword_cd_1',\
                                                              'pr_interest_keyword_cd_2','pr_interest_keyword_cd_3'\
                                                              ,'ch_interest_keyword_cd_1','ch_interest_keyword_cd_2'\
                                                              ,'ch_interest_keyword_cd_3',\
                                                                'genre_mid','cast_1','cast_2','cast_3'])
                                                                
'''
candidate_add_features = candidate.copy()
# als vector merge
candidate_add_features = pd.merge(candidate_add_features, item_factors_feature, how="left", on="album_id")
candidate_add_features = pd.merge(candidate_add_features, user_factors_feature, how="left", on="profile_id")

# interaction day, week merge
candidate_add_features = pd.merge(candidate_add_features, interaction_day_week_first, how="left",on=["profile_id","album_id"])
# candidate_add_features = pd.merge(candidate_add_features, interaction_day_week_last, how="left",on=["profile_id","album_id"])


# user cosine similarity
# train_df = pd.merge(train_df, user_cos_feature, how="left", on="profile_id")

# hour feature merge
candidate_add_features = pd.merge(candidate_add_features, hour_feature, how="left",on=["profile_id","album_id"])

In [75]:
short_con_freq_feature = short_con_freq_feature[["album_id","album_viewcount_freq"]].drop_duplicates()
candidate_add_features = pd.merge(candidate_add_features,short_con_freq_feature, how="left", on="album_id")
candidate_add_features = pd.merge(candidate_add_features,watch_df_feature, how="left",on=["profile_id","album_id"])
candidate_add_features = pd.merge(candidate_add_features, meta_df_feature, how="left",on="album_id")
candidate_add_features = pd.merge(candidate_add_features, profile_df_feature, how="left",on="profile_id")

In [76]:
# candidate_add_features = pd.merge(candidate_add_features,searched_df_feature,how="left",on=["profile_id","album_id"])
# ->> duplicates 문제 발생
candidate_add_features = pd.merge(candidate_add_features,paid_df_feature,how="left",on=["profile_id","album_id"])

In [77]:
len(candidate_add_features)

1164550

In [78]:
# 라벨 부여
train_df = pd.merge(candidate_add_features, label_df, how='left', on=['profile_id','album_id'])
train_df['rating'] = train_df['rating'].fillna(0)

# 비복원 추출 defalut
# train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)

# model

In [79]:
train_df.rating.value_counts()

0.0    1123982
1.0      40568
Name: rating, dtype: int64

In [82]:
# 타입오류 픽처 제거
train_df = train_df.drop(columns=["onair_date","title", "genre_small", "cast_1", "cast_2", "cast_3", "cast_4", "cast_5", "cast_6", "cast_7"])

# feature selection 컬럼만 학습
# train_df = train_df[['profile_id','album_id','day', 'hour_14', 'personal_counts', 'week', 'total_counts',
#        'item_vector66', 'user_vector51', 'user_vector119', 'item_vector76',
#        'user_vector59', 'item_vector97', 'user_vector164', 'user_vector196',
#        'user_vector178', 'user_vector147', 'user_vector99', 'user_vector167',
#        'item_vector165', 'user_vector75', 'user_vector50', 'user_vector136',
#        'user_vector108', 'user_vector131', 'user_vector18', 'user_vector49',
#        'user_vector62', 'user_vector152', 'user_vector177', 'item_vector119','rating']]

In [84]:
train_df

,profile_id,album_id,total_counts,general_counts,personal_counts,support,confidence,lift,item_vector0,item_vector1,...,total_time,genre_large,genre_mid,country,run_time,sex,age,age_bin,paid_label,rating
0,25844,124,1983.0,NaN,NaN,0.153531,0.801685,3.257275,0.026517,-0.060322,...,NaN,키즈,책,한국,382,M,5,유아,NaN,0.0
1,25844,16,1731.0,154.0,1.0,0.105312,0.865380,4.901661,0.007673,0.003450,...,187.0,키즈,노래율동,NaN,105,M,5,유아,NaN,0.0
2,25844,125,1610.0,141.0,NaN,NaN,NaN,NaN,-0.025760,-0.043431,...,NaN,키즈,책,한국,520,M,5,유아,NaN,0.0
3,25844,241,1581.0,184.0,1.0,NaN,NaN,NaN,0.000578,0.022174,...,399.0,키즈,책,한국,398,M,5,유아,NaN,0.0
4,25844,65,1543.0,NaN,NaN,NaN,NaN,NaN,0.029779,-0.006114,...,NaN,키즈,책,한국,536,M,5,유아,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1164545,9671,2206,NaN,NaN,NaN,NaN,NaN,NaN,-0.011045,-0.010615,...,NaN,키즈,노래율동,NaN,95,M,4,유아,NaN,0.0
1164546,9671,370,NaN,NaN,NaN,NaN,NaN,NaN,-0.005035,0.008224,...,NaN,키즈,TV만화,중국,616,M,4,유아,NaN,0.0
1164547,9671,1447,NaN,NaN,NaN,NaN,NaN,NaN,-0.000351,0.012731,...,NaN,키즈,노래율동,NaN,124,M,4,유아,NaN,0.0
1164548,9671,962,NaN,NaN,NaN,NaN,NaN,NaN,-0.011081,-0.018909,...,NaN,키즈,노래율동,한국,200,M,4,유아,NaN,0.0


In [83]:
lgbmrank = LGBMRank(train_df, mode='week', model_params={'n_estimators':5})
model, feature_importances_df = lgbmrank.train()
X_train, sample_sumbission = lgbmrank.valid_evaluation()

                 feature_importances
total_counts             1056.264904
general_counts              0.000000
personal_counts        291630.056812
support                     0.000000
confidence                  0.000000
...                              ...
run_time                    0.000000
sex                         0.000000
age                         0.000000
age_bin                     0.000000
paid_label                  0.000000

[467 rows x 1 columns]
week performance
lgbm ndcg: 0.20029421649092014


In [ ]:
# cold user MP로 채우기 
MP_list = MP_cand.album_id.unique()

sample_sumbission_cold = sample_sumbission.copy()
sample_sumbission_cold['album_id'] = sample_sumbission_cold['album_id']\
                                        .apply(lambda x: list(dict.fromkeys(np.append(x, MP_list)))[:n])

In [ ]:
ndcg_calculator(sample_sumbission_cold, test_answer_week, n)

# Evaluation

### experiment
- num_leaves= 20, learning_rate=0.005, n_estimators:5
- 어느 cand 중요한 지 판단
- popular_articles_cand :(10,10) , personal_MP_candidate:(5), genre_candidate: max(10,10)

### week
- week ndcg score : 0.05611031122402796
- popular_articles_cand week: ndcg 0.028022766659891125 (total unique item: 12)
- personal_MP_candidate week: ndcg 0.02896244104871668  (total unique item: 4303)
- genre_candidate week:       ndcg 0.01579135161487282  (total unique item: 157)
- (popular_articles_cand, personal_MP_candidate) week: ndcg 0.05206722084683086  (total unique item: 4303)
- (popular_articles_cand, genre_candidate) week: 0.03809584406788012 (total unique item: 161)
- (personal_MP_candidate, genre_candidate) week: 0.041085241338965385 (total unique item: 4338)

### month
- month ndcg score : 0.08036130090004782
- popular_articles_cand month: ndcg 0.05945057771810242 (total unique item: 15)
- personal_MP_candidate month: ndcg 0.028477345633295483  (total unique item: 2826)
- genre_candidate       month: ndcg 0.017431233023063257  (total unique item: 150)
- (popular_articles_cand, personal_MP_candidate) month: ndcg 0.05206722084683086  (total unique item: 2826)
- (popular_articles_cand, genre_candidate) month: 0.0680887232688415 (total unique item: 155)
- (personal_MP_candidate, genre_candidate) month: 0.041085241338965385 (total unique item: 2872)

In [ ]:
def evaluation(
            X_train:pd.DataFrame(), 
            sumbission:pd.DataFrame(), 
            n:int, 
            MP_cand:pd.DataFrame()
            )->pd.DataFrame():
    
    MP_list = MP_cand.album_id.unique()
    
    # each user pred 25 items
    lgbm_sub_df = X_train.sort_values(by='pred', ascending=False).groupby('profile_id').head(n)
    lgbm_user_items_dict = lgbm_sub_df.groupby('profile_id')['album_id'].unique().to_dict()
    sumbission['predicted_list'] = sumbission['profile_id']\
                                            .apply(lambda x: lgbm_user_items_dict.get(x, []))
    
    # cold start user file MP_list top25
    sumbission_cold = sumbission.copy()
    sumbission_cold['predicted_list'] = sumbission_cold['predicted_list']\
                                            .apply(lambda x: list(dict.fromkeys(np.append(x, MP_list)))[:n])
    
    return sumbission, sumbission_cold

In [ ]:
submission = pd.read_csv(path + 'sample_submission.csv')
sumbission_pred, sumbission_cold = evaluation(X_train, submission, n, MP_cand)

In [ ]:
set_list = {}
for list_i in sumbission_cold.predicted_list:
    set_list[str(list_i)] = 0

In [ ]:
print('같은 추천을 받은 유저 수:', 8311 - len(set_list))

In [ ]:
sumbission_cold

In [ ]:
# 제출 조건 충족 확인
assert submission.profile_id.nunique() == sumbission_cold.profile_id.nunique()
for pred_list in sumbission_cold.predicted_list:
    assert len(pred_list) == 25

In [ ]:
sumbission_cold

In [ ]:
sumbission_cold.to_csv('lgbm_candidate_submission_feature_add_ver8.csv', index=False)